In [1]:
import sys
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4, compact=True)
from ast import literal_eval as make_tuple
import joblib

In [2]:
##################################################################
# Parameters
MAX_unknown_CpG = 1
# features
CpG_targets = ['CpG2', 'FAM38A', 'PRKACA', 'FAM65B']
# CpG_targets = ['AML-CpG2', 'AML-CpG3', 'AML-CpG4', 'AML-CpG1']
CpG_rename = {'CpG2':'cg22797031', 
              'FAM38A':'cg27630153', 
              'PRKACA':'cg19586199', 
              'FAM65B':'cg15289427'}
# paths
DIR_methpat = "/projects/AML/code_repo/BBASeq/data_processing/processed_data/bismark/res_methpat"
# List_sample = "/projects/AML/code_repo/BBASeq/data_processing/samples.csv"
List_sample = "/home/joseph/Downloads/mrdsupport/latest_figures.csv"
CpG_posi = "/projects/AML/code_repo/BBASeq/data_processing/reference/CpG_positions.csv"

In [6]:
##################################################################
# Sample table
# samples = pd.read_csv(List_sample, header=0, usecols=range(271))
samples = pd.read_excel("/home/joseph/Downloads/mrdsupport/latest_figures.xlsx")
samples = samples.reset_index(drop=True)
print(samples.head())

   Unnamed: 0 Disease status      ID  Individual            Group Group ID  \
0         103            AML  FD_11a  Patient 13  First diagnosis       FD   
1         104            AML  FD_14a  Patient 16  First diagnosis       FD   
2         105            AML  FD_15a  Patient 17  First diagnosis       FD   
3         106            AML  FD_17a  Patient 19  First diagnosis       FD   
4         155            AML  MU_17b  Patient 19       MRD unknow       MU   

   MiSeq run New ID Original ID               FASTQ file name  ...  \
0          2    11a         11a  11a_S13_L001_R*_001.fastq.gz  ...   
1          2    14a         14a  14a_S15_L001_R*_001.fastq.gz  ...   
2          2    15a         15a  15a_S17_L001_R*_001.fastq.gz  ...   
3          2    17a         17a  17a_S19_L001_R*_001.fastq.gz  ...   
4          2    17b         17b  17b_S20_L001_R*_001.fastq.gz  ...   

  TRAK (TSH-R AK) Transferrin Triglyceride Triglyzeride Troponin T Troponin-I  \
0             nan         nan

In [7]:
##################################################################
# Load CpG position
CpG_positions = {}
CpG_num = {}
with open(CpG_posi) as f:
    for line in f:
        l = line.strip().split()
        if l[0] in CpG_rename.keys():
            CpG_positions[CpG_rename[l[0]]] = l[1:]
            CpG_num[CpG_rename[l[0]]] = len(l[1:])
        else:
            CpG_positions[l[0]] = l[1:]
            CpG_num[l[0]] = len(l[1:])


CpG_positions['cg27630153'].remove("134")
CpG_num['cg27630153'] = CpG_num['cg27630153'] - 1 

pp.pprint(CpG_positions)
pp.pprint(CpG_num)

{   'LAG3': [   '46', '58', '62', '68', '74', '92', '103', '106', '109', '135',
                '138', '145', '155', '164', '174', '179'],
    'PDE4C': [   '23', '26', '57', '63', '65', '69', '91', '105', '114', '117',
                 '126', '138', '148', '154', '164', '166', '174', '176', '193',
                 '195', '199', '214', '216', '230', '246', '257'],
    'cg15289427': [   '41', '89', '91', '102', '117', '121', '145', '175',
                      '189', '207', '220', '250', '252', '259'],
    'cg19586199': ['50', '58', '87', '97', '143', '216', '230', '232', '238'],
    'cg22797031': [   '42', '46', '64', '68', '78', '84', '90', '100', '141',
                      '169'],
    'cg27630153': [   '155', '158', '166', '176', '180', '183', '204', '208',
                      '220', '242', '249', '254', '261', '275', '304']}
{   'LAG3': 16,
    'PDE4C': 26,
    'cg15289427': 14,
    'cg19586199': 9,
    'cg22797031': 10,
    'cg27630153': 15}


In [9]:
##################################################################
# Load Methylation data
list_CpGs = {}

shift = 2

for target in CpG_targets:
    print(target)
    list_CpGs[CpG_rename[target]] = []

    for i, name in enumerate(samples["ID"]):
        print(".", end="")
        dd = pd.read_csv(DIR_methpat+"/"+name+".tsv", header=0, sep="\t", engine="c",
                         names=["amplicon", "chr", "start", "end", "pattern", "count", "sites"])
        
        for index, row in dd.iterrows():
            if row["amplicon"] == target:
                ss = make_tuple(row["sites"])
                
                entry = {}
                for site in CpG_positions[CpG_rename[target]]:
                    entry[int(site)] = "-"
                    
                for x in ss:
                    if x[0] - shift in entry.keys():
                        entry[x[0] - shift] = int(x[1])
                pp = "".join([str(xx) for xx in list(entry.values())])
#                 print(pp)
                if "-" not in pp:
                    newline = [samples["Group ID"][i], name, pp] + list(entry.values())

                    for j in range(int(row["count"])):
                        list_CpGs[CpG_rename[target]].append(newline)

    list_CpGs[CpG_rename[target]] = pd.DataFrame(np.array(list_CpGs[CpG_rename[target]]),  columns = ["group", "id", "pattern"] + CpG_positions[CpG_rename[target]])
#     list_CpGs[target] = list_CpGs[target][~list_CpGs[target].pattern.str.contains("-")]
    list_CpGs[CpG_rename[target]].iloc[:,3:] = list_CpGs[CpG_rename[target]].iloc[:,3:].astype(int)
    print()

CpG2
...........................................................................................................................................................................................................................
FAM38A
...........................................................................................................................................................................................................................
PRKACA
...........................................................................................................................................................................................................................
FAM65B
...........................................................................................................................................................................................................................


In [10]:
no_ids = []
for j, l in enumerate(samples["ID"]):
    for i, target in enumerate(CpG_targets):
        print(".", end="")
        d = sum(list_CpGs[CpG_rename[target]]["id"]==l)
        if d < 50:
            no_ids.append(l)
print(no_ids)
for l in no_ids:
    samples = samples.loc[samples["ID"] != l,]

............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................[]


In [11]:
##################################################################
# Pattern Frequency

list_patterns = {}

for target in CpG_targets:
    table_pp = list_CpGs[CpG_rename[target]].iloc[:,0:3]
#     table_pp = table_pp[~table_pp.pattern.str.contains("-")]
    table_ppc = pd.DataFrame({'counts': table_pp.groupby(['group','id'])['pattern'].value_counts()}).reset_index()
    table_sum = pd.DataFrame({'total_reads': table_pp.groupby(['group','id']).size()}).reset_index()
    table_ppc = pd.merge(table_ppc, table_sum, on=['group', 'id'])
    table_ppc["proportion"] = table_ppc["counts"] / table_ppc["total_reads"]
#     print(table_ppc.head())
    list_patterns[CpG_rename[target]] = table_ppc
    

In [12]:
##################################################################
# CpG Frequency
list_each_cpg = {}

for target in CpG_targets:
    table_m = list_CpGs[CpG_rename[target]]
    table_pp = table_m.drop("pattern", axis='columns')
    table_ppc = table_pp.groupby(['group','id']).sum().reset_index()

    table_sum = pd.DataFrame({'total_reads': table_pp.groupby(['group','id']).size()}).reset_index()
    table_ppc.iloc[:,2:] = table_ppc.iloc[:,2:].div(table_sum["total_reads"], axis=0)

    list_each_cpg[CpG_rename[target]] = table_ppc


In [13]:
colors = {'Ctrl':"#2e2e2e",
          'Training':"#949494",
          'FD':"#ac0900",
          'MP':"#ff9136", 
          'MN':"#0985fd", 
          'MU':"black", 
          'CD':"#FFFF33",
          'CL':"#A65628", 
          'DD':"#F781BF",
          'TR':"#4DAF4A",
          }
CpG_targets = ['cg15289427', 'cg22797031', 'cg27630153', 'cg19586199']

In [14]:
##################################################################
# Dump variables
import joblib
joblib.dump(samples, 'samples.sav') 
joblib.dump(list_CpGs, 'list_CpGs.sav')
joblib.dump(list_patterns, 'list_patterns.sav')
joblib.dump(list_each_cpg, 'list_each_cpg.sav') 
joblib.dump(CpG_positions, 'CpG_positions.sav')
joblib.dump(CpG_num, 'CpG_num.sav') 
joblib.dump(colors, 'colors.sav') 
joblib.dump(CpG_targets, 'CpG_targets.sav') 

['CpG_targets.sav']